In [1]:
%load_ext tensorboard

import torch
import torchvision
import torchvision.datasets as datasets
from torchvision import transforms as T
import timm
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import numpy as np
import os

In [40]:
# Adapted from https://github.com/tjmoon0104/pytorch-tiny-imagenet/blob/master/AlexNet.ipynb
data_transforms = {
    'train': T.Compose([
        T.Resize(299), T.ToTensor(),
    ]),
    'val': T.Compose([
        T.ToTensor(),
    ]),
    'test': T.Compose([
        T.ToTensor(),
    ])
}

data_dir = "C:\\Users\\Gabriel\\Documents\\Winter2022\\CIS472\\Project\\tiny-imagenet-200"
num_workers = {
    'train' : 4,
    'val'   : 0,
    'test'  : 0
}
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val','test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             num_workers=num_workers[x])
              for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}

In [41]:
#
# Adapted from Stephen's ModelExamples
#
train = dataloaders["train"]

dataiter = iter(train)
imgs, labels = next(dataiter)
img_grid = torchvision.utils.make_grid(imgs)
#atplotlib_imshow(img_grid)

In [10]:
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [43]:
inception_v3 = torchvision.models.inception_v3()
writer = SummaryWriter("InceptionV3")
writer.add_image("TinyImageNet", img_grid)
writer.add_graph(inception_v3, imgs)

%tensorboard --logdir inceptionv3

Reusing TensorBoard on port 6006 (pid 11892), started 0:01:11 ago. (Use '!kill 11892' to kill it.)